In [1]:

import cv2
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
def img_norm(img):
    new_img = img.copy().astype('float64')
    new_img -= img.min()
    max_val = new_img.max()
    new_img = new_img / (max_val) * 255
    return np.clip(new_img.round(), 0, 255).astype('uint8')

In [3]:
def img_to_RGB(img):
    b = img[:, :, 0]
    g = img[:, :, 1]
    r = img[:, :, 2]
    return r, g, b

In [4]:
def img_to_CMY(img):
    Y = 255 - img[:, :, 0]
    M = 255 - img[:, :, 1]
    C = 255 - img[:, :, 2]
    return C, M, Y

In [5]:
def img_to_HSI(img):
    new_img = img.copy().astype('float64')
    r, g, b = img_to_RGB(new_img)
    up = 0.5 * (2 * r - g - b) + 1e-12
    down = np.sqrt(np.square(r - g) + (r - b) * (g - b)) + 1e-12
    theta = np.rad2deg(np.arccos(up / down))
    H = theta
    H[b > g] = 360 - theta[b > g]
    H = H / 360 * 255
    S = (1 - 3 * (np.min(new_img, axis = 2) + 1e-12) / (r + g + b + 1e-12)) * 255
    I = (r + g + b) / 3
    return H.round().astype('uint8'), S.round().astype('uint8'), I.round().astype('uint8')


In [6]:
img = cv2.imread('../img/Fig0638(a)(lenna_RGB).tif')
img1 = cv2.imread('../img/color.jpg')
img_r, img_g, img_b = img_to_RGB(img)
img_c, img_m, img_y = img_to_CMY(img)
img_h, img_s, img_i = img_to_HSI(img1)


In [14]:
cv2.imshow("rgb", img)
cv2.imshow("r", img_r)
cv2.imshow("g", img_g)
cv2.imshow("b", img_b)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [19]:
cv2.imshow("rgb", img)
cv2.imshow("c", img_c)
cv2.imshow("m", img_m)
cv2.imshow("y", img_y)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [193]:
cv2.imshow("rgb", img1)
cv2.imshow("H", img_h)
cv2.imshow("S", img_s)
cv2.imshow("I", img_i)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
def intensitySlicing(img):
    height, width = img.shape
    result = np.zeros((height, width, 3), np.uint8)
    for i in range(height):
        for j in range(width):
            if img[i, j] > 224:
                result[i, j] = [255, 255, 255]
            elif img[i, j] > 192:
                result[i, j] = [219, 226, 255]
            elif img[i, j] > 160:
                result[i, j] = [184, 197, 255]
            elif img[i, j] > 128:
                result[i, j] = [148, 168, 255]
            elif img[i, j] > 96:
                result[i, j] = [219, 226, 255]
            elif img[i, j] > 64:
                result[i, j] = [112, 138, 255]
            elif img[i, j] > 32:
                result[i, j] = [76, 109, 255]
            else:
                result[i, j] = [41, 80, 255]

    return result

In [9]:
img2 = cv2.imread("/Users/zhuole/Desktop/DIP/DIP-python-main/images/yiqin_gray.jpg", 0)
cv2.imshow("img2", img2)
cv2.imshow("color_img2", intensitySlicing(img2))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
def grayToColor(img):
    height, width = img.shape
    result = np.zeros((height, width, 3), np.uint8)
    result[:, :, 2] = np.sin(img) * 255
    result[:, :, 1] = np.sin(img + np.pi / 3) * 255
    result[:, :, 0] = np.sin(img + np.pi / 5) * 255
    return result

In [16]:
img3 = cv2.imread("/Users/zhuole/Desktop/DIP/Chapter6/img/Fig0621(a)(weld-original).tif", 0)
cv2.imshow("img3", img3)
cv2.imshow("color_img3", grayToColor(img3))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [75]:
def colorSlicing(img, C, type):
    height, width, channels = img.shape
    result = img.copy().astype('float64') / 255
    img = img.astype('float64') / 255
    r = np.array([0.1922, 0.1608, 0.6863])
    if type == 1:
        mask_1 = np.fabs(img[:, :, 0] - r[0]) > (C / 2)
        mask_1 = np.array([mask_1] * 3).transpose(1, 2, 0)
        result[mask_1] = 0.5
        mask_1 = np.fabs(img[:, :, 1] - r[1]) > (C / 2)
        mask_1 = np.array([mask_1] * 3).transpose(1, 2, 0)
        result[mask_1] = 0.5
        mask_1 = np.fabs(img[:, :, 2] - r[2]) > (C / 2)
        mask_1 = np.array([mask_1] * 3).transpose(1, 2, 0)
        result[mask_1] = 0.5
    else:
        mask_2 = np.sum(np.square(img - r), axis=2) > C ** 2
        result[mask_2] = 0.5
    result = result * 255
    return result.astype('uint8')

In [74]:
img4 = cv2.imread("/Users/zhuole/Desktop/DIP/Chapter6/img/Fig0630(01)(strawberries_fullcolor).tif")
cv2.imshow("img4", img4)
cv2.imshow("color_img3_cube", colorSlicing(img4, 0.2549, 1))
cv2.imshow("color_img3_circ", colorSlicing(img4, 0.1765, 2))
cv2.waitKey(0)
cv2.destroyAllWindows()

(666, 666)


In [88]:
def sobelFilter(img):
    height, width, channels = img.shape
    padding = 1
    gx = np.zeros((height, width, channels))
    gy = np.zeros((height, width, channels))
    pad_img = np.zeros((height + 2 * padding, width + 2 * padding, channels))
    pad_img[padding: padding + height, padding: padding + width, :] = img
    filter_x = [[-1, -2, -1], [0, 0, 0], [1, 2, 1]]
    filter_y = [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]
    for i in range(height):
        for j in range(width):
            for k in range(channels):
                gx[i, j, k] = np.fabs(np.sum(filter_x * pad_img[i: i + 3, j: j + 3, k]))
                gy[i, j, k] = np.fabs(np.sum(filter_y * pad_img[i: i + 3, j: j + 3, k]))
    return gx, gy

In [115]:
def edgeDetection(img):
    gx, gy = sobelFilter(img)
    gxx = np.square(gx).sum(axis=2)
    gyy = np.square(gy).sum(axis=2)
    gxy = np.sum(gx * gy, axis=2)
    theta = 0.5 * np.arctan(2 * gxy / (gxx - gyy + 1e-12))
    f = np.sqrt(0.5 * ((gxx + gyy) + (gxx - gyy) * np.cos(np.rad2deg(2 * theta)) + 2 * gxy * np.sin(np.rad2deg(2 * theta))))
    print(f.shape)
    return img_norm(f)


In [125]:
img5 = cv2.imread("/Users/zhuole/Desktop/DIP/Chapter6/img/Fig0646(a)(lenna_original_RGB).tif")
edge = edgeDetection(img5)
sharpen_img5 = img_norm(img5.astype('float64') + edge[:, :, np.newaxis].astype('float64'))

(512, 512)


In [124]:

cv2.imshow("img5", img5)
cv2.imshow("edge",edge)
cv2.imshow("sharpen", sharpen_img5)
cv2.waitKey(0)
cv2.destroyAllWindows()